In [2]:
print('hello')

hello


In [ ]:
from pathlib import Path
from typing import Any, Dict
import yaml

def load_yaml(p: Path) -> Dict[str, Any]:
    """
    Opens a file.yaml content and load it into a variable as a dictionary
    """
    with p.open("r", encoding="utf-8") as f:
        return yaml.safe_load(f)
    
load_yaml(p=Path(".\components\prep.yaml")) 

<>:12: SyntaxWarning: invalid escape sequence '\c'
<>:12: SyntaxWarning: invalid escape sequence '\c'
C:\Users\joule\AppData\Local\Temp\ipykernel_24440\3791711231.py:12: SyntaxWarning: invalid escape sequence '\c'
  load_yaml(p=Path(".\components\prep.yaml"))


{'$schema': 'local.commandComponent',
 'name': 'prepare_iris',
 'display_name': 'PrepIris',
 'version': 1,
 'type': 'command',
 'inputs': {'input_data': {'type': 'uri_folder'}},
 'outputs': {'prep_data': {'type': 'uri_folder'}},
 'code': '../prep_src',
 'environment': 'local/python',
 'command': 'python prep.py --input_data ${{inputs.raw_data}}  --prep_data ${{outputs.prep_data}}'}

In [16]:
from sklearn.datasets import load_iris

data = load_iris(as_frame=True)
df = data.frame  # <--- déjà un DataFrame avec data+target
df.to_csv("../../data/iris.csv", index=False)

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

parent_dir = ""

# load_component(source: PathLike | str | IO | None = None, *, relative_origin: str | None = None, params_override: List[Dict] | None = None, **kwargs: Any) -> CommandComponent | ParallelComponent | PipelineComponent

# 1. Load components
prepare_data = load_component(source=parent_dir + "./prep.yml")

# 2. Construct pipeline
@pipeline()
def nyc_taxi_data_regression(pipeline_job_input):
    """NYC taxi data regression example."""
    prepare_sample_data = prepare_data(raw_data=pipeline_job_input)
    
    return {
        "pipeline_job_prepped_data": prepare_sample_data.outputs.prep_data
    }


pipeline_job = nyc_taxi_data_regression(
    Input(type="uri_folder", path=parent_dir + "./data/")
)